# Building Agents with Semantic Kernel

In this notebook, we'll explore how to build intelligent agents using the Semantic Kernel framework. We'll start with the basics and progress to building a multi-agent system.

## What are Semantic Kernel Agents?

Agents in Semantic Kernel are specialized AI assistants that can:
- Process and respond to user queries
- Use tools (functions) to interact with external systems
- Maintain context through conversation
- Be orchestrated to work together on complex tasks

![Agents in Semantic Kernel](../../assets/images/agents.png)

Let's start by setting up our environment.

In [ ]:
# Install required packages if needed
# !pip install semantic-kernel python-dotenv

In [1]:
# Import necessary libraries
import os
import json
import asyncio
from typing import Dict, Any

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory

from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

True

## 1. Setting Up Azure OpenAI

To use agents with Semantic Kernel, we need to set up a connection to an LLM service like Azure OpenAI. In this example, we'll use GitHub Models, which gives us easly access to the latest models.

In [2]:
service_id = "azure_openai"

if os.environ.get("AZURE_OPENAI_API_KEY"):
    print("Using Azure OpenAI")
    chat_completion_service = AzureChatCompletion(
        deployment_name=os.environ["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"],  
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        service_id=service_id,
    )
else:
    print("Using GitHub Inference")
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    base_url="https://models.inference.ai.azure.com"
    api_key=os.environ["GITHUB_TOKEN"]

    chat_completion_service = AzureAIInferenceChatCompletion(
        ai_model_id="gpt-4o-mini",
        api_key=api_key,
        endpoint=base_url,
        service_id=service_id,
    )

def create_kernel():
    """Create a kernel with Azure OpenAI service"""
    kernel = Kernel()
    
    # Add Azure OpenAI service
    kernel.add_service(chat_completion_service)
    
    return kernel

# Let's create our kernel
kernel = create_kernel()

Using Azure OpenAI


## 2. Creating Kernel Functions (Tools)

Kernel functions are the building blocks that agents can use to perform tasks. Let's create a simple plugin with a couple of functions:

In [3]:
class WeatherPlugin:
    """A plugin to interact with weather APIs and retrieve weather data"""
    
    @kernel_function(name="get_weather_data", description="Get weather data for a specific location")
    async def get_weather_data(self, location: str) -> str:
        """
        Stub function to get weather data for a specific location
        In a real implementation, this would call a weather API
        """
        print(f"Getting weather data for {location}")
        weather_data = {
            "temperature": 25,  # Celsius
            "cloud_cover": 0.2,  # 20%
            "irradiance": 800,  # W/m²
            "precipitation_chance": 0.1,  # 10%
        }
        return json.dumps(weather_data)

# Test our plugin
weather_plugin = WeatherPlugin()
weather_result = await weather_plugin.get_weather_data("Seattle")
print(weather_result)

Getting weather data for Seattle
{"temperature": 25, "cloud_cover": 0.2, "irradiance": 800, "precipitation_chance": 0.1}


## 3. Creating a Basic Agent

Now let's create our first agent. An agent in Semantic Kernel consists of:
- A kernel instance
- Instructions that define its behavior
- Optional plugins (tools) it can use
- Execution settings

Let's start with a basic agent:

In [4]:
# Define execution settings
execution_settings = AzureChatPromptExecutionSettings(
    service_id="azure_openai",
    max_tokens=1000,
)

# Create a basic agent
basic_agent = ChatCompletionAgent(
    kernel=kernel,
    name="basic_agent",
    instructions="You are a helpful assistant who answers questions clearly and concisely.",
    arguments=KernelArguments(settings=execution_settings)
)

# Let's test our basic agent
chat_history = ChatHistory()
chat_history.add_user_message("What is Semantic Kernel?")

response = await basic_agent.get_response(messages=chat_history)
print(response.content)

**Semantic Kernel** is an open-source SDK (Software Development Kit) developed by Microsoft that helps developers integrate AI and large language models (LLMs) like OpenAI's GPT or Azure OpenAI into their applications. Semantic Kernel provides tools for combining traditional programming with AI “skills,” enabling you to orchestrate prompts, manage context, connect to plugins, and perform complex reasoning tasks within your app.

Key features of Semantic Kernel include:
- **Skills and Plugins:** Define and reuse AI-powered functions (skills), such as summarization or translation, as well as plugins that bridge AI with external systems and APIs.
- **Prompt Chaining & Orchestration:** Easily create and manage sequences of prompts and workflows that use both code and LLMs.
- **Memory and Context:** Store and manage context or history to enable more coherent, stateful conversations with AI.
- **Extensibility:** Integrate other AI services, search engines, or custom logic with the kernel.

S

## 4. Creating a Tool-using Agent

Now let's create an agent that can use tools (kernel functions):

In [5]:
# Create a weather agent with the weather plugin
weather_agent = ChatCompletionAgent(
    kernel=kernel,
    name="weather_agent",
    instructions="""You are a weather specialist assistant. 
    When asked about weather for a location, use the get_weather_data function to retrieve weather information.
    Then explain the weather data in simple terms.
    Focus on temperature, cloud cover, and precipitation chance.""",
    arguments=KernelArguments(settings=execution_settings),
    plugins=[WeatherPlugin()]
)

# Let's test our weather agent
chat_history = ChatHistory()
chat_history.add_user_message("What's the weather like in New York?")

response = await weather_agent.get_response(messages=chat_history)
print(response.content)

Getting weather data for New York
Right now in New York, the temperature is warm at 25°C. There are just a few clouds in the sky, so it's mostly sunny. There's a very low chance of rain, only about 10%. Overall, it's a pleasant day!


## 5. Creating More Complex Plugins

Let's create more complex plugins that our agents can use:

In [6]:
class LocationPlugin:
    """A plugin to interact with geographic APIs and retrieve location information"""
    
    @kernel_function(name="get_location_info", description="Get location information")
    async def get_location_info(self, location: str) -> str:
        """
        Stub function to get location information
        In a real implementation, this would look up geographic data
        """
        print(f"Getting location information for {location}")
        location_info = {
            "latitude": 48.137154,
            "longitude": 11.576124,
            "elevation": 519,  # meters
            "timezone": "Europe/Berlin",
        }
        return json.dumps(location_info)

class SeasonalPlugin:
    """A plugin to calculate seasonal adjustments based on date and location"""
    
    @kernel_function(name="get_seasonal_adjustment", description="Get seasonal adjustment factors")
    async def get_seasonal_adjustment(self, date: str, latitude: float, longitude: float) -> str:
        """
        Stub function to calculate seasonal adjustments based on date and location
        In a real implementation, this would calculate solar angles etc.
        """
        print(f"Calculating seasonal adjustments for {date} at lat:{latitude}, long:{longitude}")
        seasonal_data = {
            "day_length": 14.2,  # hours
            "sun_angle": 65.3,  # degrees
            "seasonal_factor": 0.92,  # 92% of peak possible output
        }
        return json.dumps(seasonal_data)

# Test our new plugins
location_plugin = LocationPlugin()
location_result = await location_plugin.get_location_info("Munich")
print(f"Location result: {location_result}")

seasonal_plugin = SeasonalPlugin()
seasonal_result = await seasonal_plugin.get_seasonal_adjustment("2023-06-15", 48.137154, 11.576124)
print(f"Seasonal result: {seasonal_result}")

Getting location information for Munich
Location result: {"latitude": 48.137154, "longitude": 11.576124, "elevation": 519, "timezone": "Europe/Berlin"}
Calculating seasonal adjustments for 2023-06-15 at lat:48.137154, long:11.576124
Seasonal result: {"day_length": 14.2, "sun_angle": 65.3, "seasonal_factor": 0.92}


## 6. Building Specialized Agents

Now that we have multiple plugins, let's create specialized agents for each domain:

In [7]:
# Create specialized agents
def create_agents(kernel):
    settings = AzureChatPromptExecutionSettings(
        service_id="azure_openai",
        max_tokens=4096,
    )
    
    # Weather agent
    weather_agent = ChatCompletionAgent(
        kernel=kernel,
        name="weather_agent",
        instructions="""You are a weather data specialist. You provide weather forecasts for specific locations.
        
        When asked about weather for a location, use the get_weather_data function to retrieve weather information.
        Then explain what the weather data means for PV production in simple terms.
        
        Focus on factors like solar irradiance, cloud cover, and temperature that affect PV production.""",
        arguments=KernelArguments(settings=settings),
        plugins=[WeatherPlugin()]
    )
    
    # Location agent
    location_agent = ChatCompletionAgent(
        kernel=kernel,
        name="location_agent",
        instructions="""You are a geographical data specialist. You provide location information like latitude, longitude, and elevation.
        
        When asked about a location, use the get_location_info function to retrieve the geographical information.
        Explain how the location's position affects solar energy potential.
        
        Consider factors like latitude (affecting sun angle) and local terrain characteristics.""",
        arguments=KernelArguments(settings=settings),
        plugins=[LocationPlugin()]
    )
    
    # Seasonal agent
    seasonal_agent = ChatCompletionAgent(
        kernel=kernel,
        name="seasonal_agent",
        instructions="""You are a seasonal adjustment specialist. You calculate how the time of year affects solar energy production.
        
        When provided with a date and location coordinates, use the get_seasonal_adjustment function to determine seasonal factors.
        Explain how the season and sun position impact PV production.
        
        Consider factors like day length, sun angle, and seasonal variations that affect PV output.""",
        arguments=KernelArguments(settings=settings),
        plugins=[SeasonalPlugin()]
    )
    
    return {
        "weather": weather_agent,
        "location": location_agent,
        "seasonal": seasonal_agent
    }

# Create our specialized agents
agents = create_agents(kernel)

## 7. Testing Individual Agents

Let's test each of our specialized agents:

In [8]:
# Test location agent
location_chat = ChatHistory()
location_chat.add_user_message("Tell me about Munich, Germany as a location for solar power.")

response = await agents["location"].get_response(messages=location_chat)
print("Location Agent Response:")
print(response.content)
print("\n" + "-"*50 + "\n")

Getting location information for Munich, Germany
Location Agent Response:
Munich, Germany is located at approximately 48.14° N latitude and 11.58° E longitude, with an elevation of about 519 meters above sea level.

The solar energy potential in Munich is influenced by its mid-northern latitude. This latitude means the sun is lower in the sky compared to locations closer to the equator, especially in winter, leading to shorter daylight hours and a lower sun angle. This reduces the annual solar irradiation compared to southern European or equatorial locations.

However, Munich’s moderately high elevation can offer slightly clearer air, which may improve solar potential by reducing atmospheric absorption, though local weather (frequent clouds) can limit some of these benefits.

Overall, Munich is a viable location for solar power, with noticeable seasonal variations—higher output in summer and lower in winter due to its position on the globe. Properly angling solar panels to maximize sun

In [9]:
# Test weather agent
weather_chat = ChatHistory()
weather_chat.add_user_message("What's the weather forecast for Munich, and how will it affect PV production?")

response = await agents["weather"].get_response(messages=weather_chat)
print("Weather Agent Response:")
print(response.content)
print("\n" + "-"*50 + "\n")

Getting weather data for Munich
Weather Agent Response:
The weather forecast for Munich shows a temperature of 25°C, low cloud cover (20%), and a strong solar irradiance of 800 W/m². These conditions are very favorable for PV (solar) production:

- Low cloud cover means plenty of sunlight will reach the solar panels.
- High solar irradiance provides a strong energy source for PV systems.
- The moderate temperature (25°C) is also good, as PV panels operate efficiently without excessive heat.

Overall, you can expect high PV output in Munich under these weather conditions.

--------------------------------------------------



In [10]:
# Test seasonal agent
seasonal_chat = ChatHistory()
seasonal_chat.add_user_message("How will the seasonal factors on June 15, 2023 at latitude 48.137154, longitude 11.576124 affect PV production?")

response = await agents["seasonal"].get_response(messages=seasonal_chat)
print("Seasonal Agent Response:")
print(response.content)
print("\n" + "-"*50 + "\n")

Calculating seasonal adjustments for 2023-06-15 at lat:48.137154, long:11.576124
Seasonal Agent Response:
On June 15, 2023, at latitude 48.137154 and longitude 11.576124 (Munich, Germany), the seasonal factors influence PV production as follows:

- Day length: 14.2 hours — This is near the summer solstice, so days are long, allowing solar panels to generate electricity for more hours.
- Sun angle: 65.3° — The sun is high in the sky, resulting in more direct sunlight on PV modules, which increases their efficiency and power output.
- Seasonal factor: 0.92 — This means PV production is at 92% of its theoretical annual maximum.

Explanation:
During late spring and early summer, the combination of long days and high sun angles means solar panels receive abundant, direct sunlight. This results in strong PV generation, with only slight reductions from perfect conditions due to atmospheric effects or weather variability. Thus, PV output is excellent during this period.

----------------------

## 8. Building a Multi-Agent System

Now let's build a multi-agent system that coordinates these specialized agents to produce a comprehensive PV forecast. We'll add two more agents to process and summarize the information:

In [11]:
# Create additional agents for our multi-agent system
def create_additional_agents(kernel):
    settings = AzureChatPromptExecutionSettings(
        service_id="azure_openai",
        max_tokens=4096,
    )
    
    # PV Forecast agent
    pv_forecast_agent = ChatCompletionAgent(
        kernel=kernel,
        name="pv_forecast_agent",
        instructions="""You are a PV system specialist. You analyze all data and produce the final PV output forecast.
        
        Review the weather data, location information, and seasonal factors provided by the other agents.
        Based on this information and the system specifications, provide a comprehensive forecast of PV production.
        
        Include estimated production in kWh, efficiency factors, and confidence levels in your analysis.
        Explain how each factor (weather, location, season) contributes to your forecast.""",
        arguments=KernelArguments(settings=settings)
    )
    
    # Summary agent
    summary_agent = ChatCompletionAgent(
        kernel=kernel,
        name="summary_agent",
        instructions="""You are a summarization specialist. Your job is to take all the detailed information from the other agents and produce a clear, concise final report.
        
        Synthesize the technical details into an actionable PV production forecast that highlights:
        - Expected daily production in kWh
        - Key factors influencing the forecast
        - Confidence level in the prediction
        - Any notable considerations for the specific location and date
        
        Your summary should be comprehensive yet accessible to non-technical users.""",
        arguments=KernelArguments(settings=settings)
    )
    
    return {
        "pv_forecast": pv_forecast_agent,
        "summary": summary_agent
    }

# Add the new agents to our agents dictionary
additional_agents = create_additional_agents(kernel)
agents.update(additional_agents)

## 9. Orchestrating Multi-Agent Interaction

Now let's orchestrate the interaction between our agents to solve a complex task:

In [12]:
async def run_pv_forecast(location: str, date: str, system_specs: Dict[str, Any]) -> None:
    """Run the PV forecast using a Semantic Kernel multi-agent approach"""
    print(f"\nRunning PV forecast for {location} on {date}\n")
    
    # Create a chat history for sequential interaction
    chat = ChatHistory()
    
    # Initial task formulation
    task = f"""
    I need a PV forecast for a pv system in {location} on {date}.
    
    System specifications:
    - Capacity: {system_specs['capacity_kw']} kW
    - Panel type: {system_specs['panel_type']}
    - Efficiency: {system_specs['efficiency'] * 100}%
    - Orientation: {system_specs['orientation']}
    - Tilt angle: {system_specs['tilt_angle']} degrees
    
    Please provide a comprehensive forecast of expected PV production.
    """

    # Add the initial task to the chat history
    chat.add_user_message(task)
    
    # Step 1: Location agent gets geographical data
    print("Step 1: Location agent retrieving geographical data...")
    response = await agents["location"].get_response(messages=chat)
    chat.add_assistant_message(response.content.content)
    print(response.content)
    
    # Step 2: Weather agent gets weather forecast
    print("\nStep 2: Weather agent retrieving weather data...")
    response = await agents["weather"].get_response(messages=chat)
    chat.add_assistant_message(response.content.content)
    print(response.content)
    
    # Step 3: Seasonal agent performs seasonal analysis
    print("\nStep 3: Seasonal agent performing seasonal analysis...")
    chat.add_user_message("Based on the location information, please provide seasonal adjustment factors.")
    response = await agents["seasonal"].get_response(messages=chat)
    chat.add_assistant_message(response.content.content)
    print(response.content)
    
    # Step 4: PV forecast agent produces initial forecast
    print("\nStep 4: PV forecast agent producing initial forecast...")
    chat.add_user_message("Using the weather data, location information, and seasonal factors, provide a PV production forecast for the specified system.")
    response = await agents["pv_forecast"].get_response(messages=chat)
    chat.add_assistant_message(response.content.content)
    print(response.content)
    
    # Step 5: Summary agent produces final report
    print("\nStep 5: Summary agent producing final report...")
    chat.add_user_message("Please provide a final, concise summary of the PV forecast.")
    response = await agents["summary"].get_response(messages=chat)
    chat.add_assistant_message(response.content.content)
    print(response.content)
    
    print("\nPV forecast completed!")

In [13]:
# Run our multi-agent system with example data
system_specs = {
    "capacity_kw": 5.0,
    "panel_type": "monocrystalline",
    "efficiency": 0.22,
    "orientation": "south",
    "tilt_angle": 30,
}

await run_pv_forecast("Munich, Germany", "2023-06-15", system_specs)


Running PV forecast for Munich, Germany on 2023-06-15

Step 1: Location agent retrieving geographical data...
Getting location information for Munich, Germany
Getting weather data for Munich, Germany
Calculating seasonal adjustments for 2023-06-15 at lat:48.137154, long:11.576124
Here’s a comprehensive PV production forecast for your 5.0 kW monocrystalline solar system in Munich, Germany, on June 15, 2023:

Location Details:
- Latitude: 48.137° N 
- Longitude: 11.576° E 
- Elevation: 519 meters above sea level

How Munich’s Location Affects Solar Energy Potential:
- The relatively high latitude results in long daylight hours in June (about 14.2 hours), but the sun’s maximum angle—at 65.3°, lower than the tropics—means sunlight is slightly less direct.
- The elevation (519 m) may help slightly due to thinner air (less atmospheric attenuation).
- Seasonal adjustment factor is 0.92, meaning irradiance is strong but not quite at peak (mainly due to some atmospheric conditions typical in e

## 10. Exercise: Creating Your Own Agent

Now it's your turn! Create your own agent that can:
1. Accept user input about a task
2. Use at least one tool (kernel function)
3. Provide a helpful response

For example, you could create an agent that:
- Analyzes text sentiment
- Generates creative content
- Performs calculations
- Answers questions about a specific domain

Here's a template to get you started:

In [14]:
# Create your own plugin
class MyPlugin:
    @kernel_function(name="my_function", description="Description of what your function does")
    async def my_function(self, input_param: str) -> str:
        # Your function implementation here
        return f"Processed: {input_param}"

# Create your agent
my_agent = ChatCompletionAgent(
    kernel=kernel,
    name="my_custom_agent",
    instructions="Write detailed instructions for your agent here...",
    arguments=KernelArguments(settings=execution_settings),
    plugins=[MyPlugin()]
)

# Test your agent
my_chat = ChatHistory()
my_chat.add_user_message("Your test message here")

# Run the agent
# response = await my_agent.get_response(my_chat)
# print(response.content)

## 11. Advanced Concepts: Agent System Design

When designing multi-agent systems, consider these principles:

1. **Specialization**: Each agent should have a specific role and expertise
2. **Coordination**: Agents should pass context and build on each other's work
3. **Tool Integration**: Provide agents with relevant tools to perform tasks
4. **Prompt Engineering**: Write clear, specific instructions for each agent
5. **Error Handling**: Build in mechanisms to handle unexpected situations

You can extend the example we built by:
- Adding more specialized agents
- Implementing error handling
- Creating a dynamic orchestration system
- Storing and retrieving agent conversation history
- Adding human-in-the-loop feedback mechanisms

## 12. Conclusion

In this notebook, we've explored how to build intelligent agents using Semantic Kernel:

1. Set up a Semantic Kernel environment with Azure OpenAI.
2. Created kernel functions (tools) for agents to interact with external systems.
3. Built specialized agents with clearly defined roles and responsibilities.
4. Orchestrated multiple agents to collaboratively solve complex tasks.

Semantic Kernel agents provide a powerful framework for creating AI systems capable of reasoning, leveraging tools, and collaborating effectively to address complex problems. The Semantic Kernel agent framework simplifies the development, testing, and deployment of these intelligent systems.

For more information, visit the [Semantic Kernel documentation](https://learn.microsoft.com/en-us/semantic-kernel/).